# Python and SQLLite3
SQLite allows us to quickly get up and running with databases, without spinning up larger databases like MySQL or Postgres. 

We will be doing the following:
- connect to SQLLite
- create a database, 
- create a table, 
- insert, select, update, and delete data from a table. 

SQLLite is great for prototyping, you can then migrate your database to other DB engines such as MySQL, Postgres ...

*** NOTE: for this tutorial, I am assuming that you have basic knowledge on SQL ***

# Connecting to a database

In [ ]:
# create a connection to a database 
import sqlite3

# you can create an 'in-memory database' or a database file.
# conn = sqlite3.connect(':memory:') # in memory database
conn = sqlite3.connect('./../../data/employee.db')

NOTE: If the database does not exist, then it will be created and finally, a database object will be returned

In [ ]:
# let's create a cursor to execute SQL commands
c = conn.cursor()

In [ ]:
# if exists drop table employees
c.execute('DROP TABLE employees')

In [ ]:
conn.commit()

In [ ]:
# let's create an employees table - fields: first(name), last(name), pay
# data types: text, text, real
c.execute("""CREATE TABLE employees (
            first text,
            last text,
            pay real
            )""")


See sqlite data types: https://www.sqlite.org/datatype3.html

In [ ]:
# committing your work into the database
conn.commit() 

In [ ]:
# close the connection to the database
conn.close()

In [ ]:
# Re-open the database and set the cursor
conn = sqlite3.connect('./../../data/employee.db')
c = conn.cursor()

In [ ]:
# let's add an employee to the database
c.execute("INSERT INTO employees VALUES('Anne','Roy', 150000)")
conn.commit()

In [ ]:
# let's now query the database
c.execute("SELECT * FROM employees WHERE last = 'Roy'")
# get one row
c.fetchone()

As you can see, the results are formatted as a **list of tuples**. Each tuple corresponds to a row in the database that we accessed

In [ ]:
# let's add more users
c.execute("INSERT INTO employees VALUES('Chaka','Motley', 200000)")
c.execute("INSERT INTO employees VALUES('Paul','Doe', 198000)")
c.execute("INSERT INTO employees VALUES('Sophie','Doe', 180000)")
conn.commit()

In [ ]:
c.execute("SELECT * FROM employees WHERE last = 'Doe'")
# you can also fetch many rows
# Fetch all (remaining) rows of a query result
results = c.fetchall()
print(results)

In [ ]:
c.execute("SELECT * FROM employees")
# you can get many - i.e.: 3 rows
c.fetchmany(3)

## Selecting Rows using a For Loop

In [ ]:
for row in c.execute("SELECT * FROM employees"):
    print("FNAME = ", row[0])
    print("LNAME = ", row[1]), 
    print("INCOME = ", row[2], '\n')
#conn.commit()

## Inserting Additional Records
Once  your values are inserted for your tables. You can use Python functions to access more easily to your infos, without repeting the sql code all the time.

In [ ]:
# let's insert additional record via our class Employee
emp_1 = ('John', 'Doe', 80000)
emp_2 = ('Jane', 'Doe', 90000)

NOTE: currently these 2 above employees are just Python objects (tuple).  They are not inserted in the database yet.*

In [ ]:
# slicing the tuple list
print(emp_1[0:3])

In [ ]:
print(emp_1[0])

In [ ]:
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_1[0], 'last': emp_1[1], 'pay': emp_1[2]})
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_2[0], 'last': emp_2[1], 'pay': emp_2[2]})

In [ ]:
# commit
conn.commit()

In [ ]:
# getting the rows from the database 
result = c.execute("SELECT * FROM employees WHERE last=:last", {'last': 'Doe'})
print(c.fetchall())

In [ ]:
conn.close()

## Connecting to an  in-memory database
The database will live in the RAM.

A SQLite in-memory database's primary advantage is performance: rather than reading and writing to disk, it will keep the whole database in memory. Memory is much faster than disk. You'll see the biggest performance improvement with a spinning disk or a heavily IO loaded server, and less with an SSD.

IMPORTANT NOTE: RESTART A FRESH KERNEL INSTANCE

Let's explore the 'in-memory' database
The database will live in the RAM.
<p>***NB: retstart a fresh kernel instance***</p>

In [1]:
import sqlite3
conn = sqlite3.connect(':memory:') 
c = conn.cursor()

When this is done, no disk file is opened. Instead, a new database is created purely in memory. The database ceases to exist as soon as the database connection is closed. Every :memory: database is distinct from every other. So, opening two database connections each with the filename ":memory:" will create two independent in-memory databases.

In [2]:
# Create the employee table in memory
c.execute("""CREATE TABLE employees (
            first text,
            last text,
            pay real
            )""")

In [3]:
# let's insert additional record via our class Employee
emp_1 = ('Alice', 'Doe', 80000)
emp_2 = ('Hector', 'Doe', 90000)

In [4]:
# insert the values above into the database
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_1[0], 'last': emp_1[1], 'pay': emp_1[2]})
c.execute("INSERT INTO employees VALUES (:first, :last, :pay)", {'first': emp_2[0], 'last': emp_2[1], 'pay': emp_2[2]})

In [5]:
# commit
conn.commit()

In [6]:
# getting the rows from the database 
result = c.execute("SELECT * FROM employees WHERE last=:last", {'last': 'Doe'})
print(c.fetchall())

[('Alice', 'Doe', 80000.0), ('Hector', 'Doe', 90000.0)]


## Using Python functions to access records

In [7]:
def get_emps_by_name(lastname):
        c.execute("SELECT * FROM employees WHERE last=:last", {'last': lastname})
        return c.fetchall()


In [8]:
get_emps_by_name('Doe')

[('Alice', 'Doe', 80000.0), ('Hector', 'Doe', 90000.0)]

In [9]:
# update function
def update_pay(fname, lname, pay):
        with conn:
            c.execute("""UPDATE employees SET pay = :pay
                    WHERE first = :first AND last = :last""",
                  {'first': fname, 'last': lname, 'pay': pay})

In [10]:
# update function execution
update_pay('Alice', 'Doe', 100000.98)

In [11]:
# see the result
get_emps_by_name('Doe')

[('Alice', 'Doe', 100000.98), ('Hector', 'Doe', 90000.0)]

In [12]:
# remove function
def remove_emp(fname, lname):
        with conn:
            c.execute("DELETE from employees WHERE first = :first AND last = :last",
                  {'first': fname, 'last': lname})

In [13]:
# remove function execution
remove_emp('Alice', 'Doe')

In [14]:
# see the result
get_emps_by_name('Doe')

[('Hector', 'Doe', 90000.0)]

# Additional Resource

[SQLLite3 Documentation](https://docs.python.org/3/library/sqlite3.html)

[SQLLite3 Cheatsheet](https://digital-forensics.sans.org/media/SQlite-PocketReference-final.pdf)

[SQLLite3 FAQ](https://www.sqlite.org/faq.html)

[SQLLIte3 on line](https://sqliteonline.com/)

[dbBrowser](https://sqlitebrowser.org/)